This is a file that blah blah blah

In [ ]:
import emission.storage.timeseries.abstract_timeseries as esta

In [ ]:
import pandas as pd

In [ ]:
agg = esta.TimeSeries.get_aggregate_time_series()

In [ ]:
all_ct = agg.get_data_df("analysis/confirmed_trip")

In [ ]:
all_ct.head()

In [ ]:
confirmed_ct = all_ct[all_ct.user_input != {}]
confirmed_ct.head()

In [ ]:
confirm_only = pd.DataFrame(confirmed_ct.user_input.to_list())
confirm_only.head()

In [ ]:
confirm_only.mode_confirm.head()

In [ ]:
mode_confirm_vals = confirm_only.mode_confirm.value_counts()
mode_confirm_vals

In [ ]:
mode_confirm_vals_df = pd.DataFrame(mode_confirm_vals)
mode_confirm_vals_df

In [ ]:
total_mode_confirms = mode_confirm_vals_df.mode_confirm.sum()
mode_confirm_vals_df["quartile"] = mode_confirm_vals.apply(lambda x: x/total_mode_confirms)
large_chunk_mode_confirm = mode_confirm_vals_df.where(lambda x: x.quartile > 0.01).dropna()
large_chunk_mode_confirm

In [ ]:
misc_count = (total_mode_confirms - large_chunk_mode_confirm.mode_confirm.sum())
misc_count

In [ ]:
large_chunk_mode_confirm.loc["misc"] = pd.Series({"mode_confirm": misc_count , "quartile": misc_count/total_mode_confirms})

In [ ]:
large_chunk_mode_confirm

In [ ]:
ax = large_chunk_mode_confirm.mode_confirm.plot.pie()

In [ ]:
ax.figure.savefig("/plots/mode_confirm.png")

In [ ]:
small_chunk_mode_confirm = mode_confirm_vals_df.where(lambda x: x.quartile < 0.01).dropna()
small_chunk_mode_confirm

In [ ]:
ax = small_chunk_mode_confirm.mode_confirm.plot.pie()

In [ ]:
ax.figure.savefig("/plots/mode_confirm_misc_expand.png")